# Portfolio Simulator

When training machine learning models it is essential to perform cross validation to ensure we're not overfitting. To that end, I have constructed a portfolio simulator to test our models on loans they have not seen before.

The portfolio simulator works as follows: {UPDATE}

The necessary classes and functions are contained in the `src/portfolio.py` file.

In [1]:
import pandas as pd
import numpy as np
import datetime
from src.portfolio import *

In [2]:
payments = pd.read_pickle('data/df_payments_cleaned_all.pkl.bz2', compression='bz2')
payments.tail()

RECEIVED_AMT_INVESTORS  PBAL_END_PERIOD_INVESTORS  \
RECEIVED_D LOAN_ID                                                        
2020-08-01 169200115              840.020020               34773.839844   
           169216811              217.649994                1819.479980   
           169224081              652.979980               19557.210938   
           169225459             1394.140015                1560.839966   
           169232425             8095.520020                   0.000000   

                     IssuedDate  mths_since_issue  
RECEIVED_D LOAN_ID                                 
2020-08-01 169200115 2020-06-01                 2  
           169216811 2020-06-01                 2  
           169224081 2020-06-01                 2  
           169225459 2020-06-01                 2  
           169232425 2020-06-01                 2

Next let's read in the predictions that are trained models have already made for the loans in the testing set.

In [3]:
predictions_xgb = pd.read_pickle('data/model_xgb_predictions.pkl.bz2')
predictions_random = pd.read_pickle('data/model_random_pick.pkl.bz2')
predictions_high_interest = pd.read_pickle('data/model_high_interest_rate.pkl.bz2')
predictions_low_interest = pd.read_pickle('data/model_low_interest_rate.pkl.bz2')

In [ ]:
min_rois = np.linspace(5, 11, num = 51)
rois = []
balances = []

for roi in min_rois:

    myPortfolio = Portfolio(20000, 25, datetime.date(2017,8,1), predictions_xgb, payments, min_roi=roi)
    while myPortfolio.date < datetime.date(2020,7,1):
        myPortfolio.simulate_month()
    rois.append(roi)
    balances.append(myPortfolio.total_balance)